This is a tutorial from Arcee and AI Makerspace on how to train an end to end RAG model jointly with retriever and generator to create a Domain Adapted Language Model (a DALM). E2E RAG training allows you to create performant contextualized retrievers for your RAG system.

The model we train will learn how to retrieve documents from our context documents and generate responses in a unified system.

We will:

* Clone and Install DALM dependencies
* Prepare a toy dataset
* Run E2E RAG training for our DALM
* Run inference on our trained DALM
* Export our trained retriever model
* Query example DALMs at https://app.arcee.ai

![e2erag](https://i.imgur.com/0uMWN8H.png)




In [ ]:
#Make sure you have a GPU enabled
!nvidia-smi

Sat Sep 23 09:36:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Install the DALM repo `indomain`

We will install DALM package https://github.com/arcee-ai/DALM from pypi!

**DALM**




In [1]:
!pip install -qqq indomain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2

# Prepare Dataset of Examples

E2E RAG requires a dataset of [passage, question, answer] triples

At inference time, our model will take a users query, draw from the available passages, and pass relevant context to the generator to create an answer.

## Toy Dataset

For this tutorial we will train a simple toy dataset to show how the E2E RAG training can be accomplished. To build your own DALM, you should train on your own documents

## Generating Q+A Pairs

If you do not have labeled triples, you can generate QA pairs with:

```
dalm qa-gen path/to/dataset.csv
```

for example to generate QA pairs with the "taesiri/arxiv_qa" hugging face dataset:

```
!dalm qa-gen "taesiri/arxiv_qa" --output-dir qa-outputs --passage-column-name text --title-column-name title
```

For more information, you can run `dalm qa-gen --help`



In [2]:
# Load our toy dataset
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/arcee-ai/DALM/main/dalm/datasets/toy_data_train.csv")
df.to_csv("dataset.csv")
df.head()

,Question,Abstract,Answer
0,Photosynthesis definition,Process where plants convert light into energy...,Energy conversion in plants
1,"Author of ""Romeo and Juliet""",Famous play written by William Shakespeare.,William Shakespeare
2,Capital of France,The capital city of France is Paris.,Paris
3,Declaration of Independence date,"Document signed on July 4, 1776 declaring Amer...",1776
4,World's largest ocean,The Pacific Ocean is the largest and deepest o...,Pacific Ocean


# Train DALM model with gpt-neo-125m generator and bge-large-en retriever

For this demo notebook, we will train a small 125 million GPT Neo model and we will train the small bge retriever model.

For better results, you will want to substitute `meta-llama/Llama-2-7b` and `bge-large-en`. You will need a bigger GPU (like A10080GB) to train this - and make sure to adjust batch size to saturate your GPU memory


In [3]:
!dalm train-rag-e2e \
"./dataset.csv" \
"BAAI/bge-large-en" \
"EleutherAI/gpt-neo-125m" \
--output-dir "./rag_e2e_checkpoints" \
--with-tracking \
--report-to all \
--per-device-train-batch-size 1

2023-10-04 19:52:15.681654: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
10/04/2023 19:52:43 - INFO - dalm.training.rag_e2e.train_rage2e - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

Extracting data files: 100% 1/1 [00:00<00:00, 1626.96it/s]
Generating train split: 19 examples [00:00, 1087.85 examples/s]
Running tokenizer on dataset: 100% 19/19 [00:00<00:00, 1160.42 examples/s]
10/04/2023 19:52:49 - INFO - dalm.training.rag_e2e.train_rage2e - Sample 3 of the training set: {'retriever_query_input_ids': [101, 1001, 23032, 1001, 8170, 1997, 4336, 3058, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'retriever_query_token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

# Evaluate Retriever Training

Here we will evalauate our contextualized retriever against our passage data that we will have in our database at retrieval time

In [10]:
!dalm eval-rag "./dataset.csv" \
  --retriever-name-or-path "BAAI/bge-large-en" \
  --generator-name-or-path "EleutherAI/gpt-neo-125m" \
  --passage-column-name Abstract \
  --query-column-name Question \
  --answer-column-name Answer \
  --evaluate-generator \
  --query-batch-size 5 \
  --retriever-peft-model-path ./rag_e2e_checkpoints/retriever \
  --generator-peft-model-path ./rag_e2e_checkpoints/generator

2023-10-04 19:56:18.622678: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Running tokenizer on dataset (num_proc=4): 100% 19/19 [00:00<00:00, 40.43 examples/s]
Filter: 100% 19/19 [00:00<00:00, 187.35 examples/s]
Starting to generate passage embeddings (Number of passages: 19)
100% 3/3 [00:00<00:00,  7.81it/s]
Construct passage index
Evaluation start
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:50256 fo

# Query Example DALM

In this tutorial, we showed how to train a retriever model with a small generator. To see a DALM in production, we can query DALM models on https://app.arcee.ai to get an idea for their behavior at scale.

For example, we can query `DALM-Patent` on https://app.arcee.ai that has been trained on the USPTO Patent database

![infergif](https://arcee-public.s3.us-east-2.amazonaws.com/infer.gif)
